In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_0_0001_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 0.0001
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 99.5254, Epoch Time 1681.1855(1681.1855), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7075(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 12.6726(13.6710) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1510(19.3613) | Error 0.6956(0.7077) Steps 550(523.97) | Grad Norm 8.7338(6.0051) | Total Time 0.00(0.00)
Iter 0120 | Time 13.8107(13.6052) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.8145(18.9691) | Error 0.6867(0.7049) Steps 544(528.02) | Grad Norm 3.5186(6.0690) | Total Time 0.00(0.00)
Iter 0130 | Time 13.4047(13.5958) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7593(18.6680) | Error 0.6711(0.7004) Steps 556(533.07) | Grad Norm 4.4387(6.6069) | Total Time 0.00(0.00)
Iter 0140 | Time 14.0782(13.6069) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7059(18.4326) | Error 0.6378(0.6931) Steps 526(537.44) | Grad Norm 4.1644(6.7688) | Total 

...

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0005 | Time 76.6991, Epoch Time 1498.8367(1659.7084), Bit/dim 4.9592(best: 5.2222), Bit/dim Marginal 4.9157(best: 5.1794), Xent 1.9312, Loss 5.9248, Error 0.6751(best: 0.6684)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0520 | Time 14.0335(13.7323) | Bit/dim 4.8726(4.9903) | Xent 1.5048(1.5957) | Loss 13.0156(13.8484) | Error 0.5356(0.5509) Steps 538(549.88) | Grad Norm 21.4122(26.4405) | Total Time 0.00(0.00)
Iter 0530 | Time 13.6198(13.8476) | Bit/dim 4.8576(4.9636) | Xent 1.4662(1.5720) | Loss 12.7977(13.6132) | Error 0.4933(0.5436) Steps 544(551.70) | Grad Norm 18.9521(24.7407) | Total Time 0.00(0.00)
Iter 0540 | Time 13.6624(13.8734) | Bit/dim 4.9710(4.9527) | Xent 1.5334(1.5796) | Loss 13.0863(13.4939) | Error 0.5200(0.5457) Steps 520(554.38) | Grad Norm 52.4004(32.0512) | Total Time 0.00(0.00)
Iter 0550 | Time 14.5100(13.9205) | Bit/dim 4.8747(4.9511) | Xent 1.6415(1.6325) | Loss 13.0769(13.4523) | Error 0.5578(0.5652) Steps 562(555.73) | Grad Norm 17.1542(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0006 | Time 83.5678, Epoch Time 1607.3236(1658.1368), Bit/dim 4.7701(best: 4.9592), Bit/dim Marginal 4.7122(best: 4.9157), Xent 1.9258, Loss 5.7330, Error 0.6720(best: 0.6684)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0620 | Time 14.1273(14.3263) | Bit/dim 4.7546(4.7926) | Xent 1.5665(1.5359) | Loss 12.8651(13.4320) | Error 0.5556(0.5390) Steps 580(563.60) | Grad Norm 50.0892(26.4380) | Total Time 0.00(0.00)
Iter 0630 | Time 15.1045(14.4846) | Bit/dim 4.7498(4.7804) | Xent 1.5119(1.5315) | Loss 12.7271(13.2513) | Error 0.5133(0.5372) Steps 574(568.70) | Grad Norm 22.6564(27.3433) | Total Time 0.00(0.00)
Iter 0640 | Time 14.4888(14.5183) | Bit/dim 4.7287(4.7653) | Xent 1.4568(1.5212) | Loss 12.6194(13.0867) | Error 0.5311(0.5329) Steps 580(571.01) | Grad Norm 21.6566(28.1861) | Total Time 0.00(0.00)
Iter 0650 | Time 14.5370(14.5517) | Bit/dim 4.6951(4.7538) | Xent 1.4259(1.5075) | Loss 12.5811(12.9679) | Error 0.4844(0.5282) Steps 592(574.23) | Grad Norm 15.9194(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,